In [ ]:
import jphot as jp
import lya_utils as lya
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline 

In [ ]:
p = jp.photonlist('PhotICs.dat','test.dat')
nphot_per_packet = p.nRealPhotons / p.nphoton # nb of real phot /s / MC phot
lbda = lya.clight / p.nu * 1e8  # [A]
ener = nphot_per_packet * lya.h_cgs * p.nu_ic  # [erg/s/MC phot]
h,edges = np.histogram(lbda,bins=500,weights=ener)
bin_centers = 0.5*(edges[:-1]+edges[1:])
dlbda = bin_centers[1]-bin_centers[0]
h = h / dlbda # erg/s/A

In [ ]:
plt.plot(bin_centers,h)
plt.xlim(1189,1198)
plt.ylim(0,6e-12)
plt.xlabel(r'$\lambda \ [\AA]$',fontsize=15)
plt.ylabel(r'$F_\lambda$',fontsize=15)